In [1]:
from optbinning import OptimalBinning


from os import listdir
from os.path import isfile, join
from datetime import datetime, timedelta
import time
import sys

import pandas as pd
import numpy as np
import random

from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

from optbinning import OptimalBinning
from catboost import CatBoostClassifier, Pool, cv

import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from bayes_opt import BayesianOptimization
import catboost

In [2]:

i = 'dataset1.csv'
df = pd.read_csv(i, sep=';', low_memory=False, error_bad_lines=False)
df_view = df.copy()

In [3]:
print(df.shape)
#df.columns
df_types = df_view.dtypes
df_types.value_counts()

(99976, 43)


int64      20
float64    19
object      3
bool        1
dtype: int64

In [4]:
categorical_cols = df_types[df_types=='object']
categorical_cols = list(categorical_cols.index)
categorical_cols = [x for x in categorical_cols if x!='uuid']
categorical_cols

['x_13', 'x_14', 'x_18']

In [6]:
numerical_cols = df_types[df_types!='object']
numerical_cols = list(numerical_cols.index)
numerical_cols = [x for x in numerical_cols if x!='default']

numerical_cols = list(set(numerical_cols)-set(ordinal_cols))

In [7]:
df_view= df_view[df_view.default.isnull()==False]
random.seed(1)
df_view['random'] = np.random.randint(1, df_view.shape[0], df_view.shape[0])/df_view.shape[0]
porc_train=0.6
porc_test=0.2

df_view['flag_train'] = 0 #train
df_view['flag_train'] = ((df_view['random'] > porc_train)).astype('int') #eval
df_view.loc[df_view['random'] > (1-porc_test),'flag_train']=2 #test

df_view.flag_train.value_counts()

0    54122
2    18084
1    17770
Name: flag_train, dtype: int64

In [8]:
cuts_bivar_cat={}
for var in categorical_cols + ordinal_cols:
  print('var: ' + str(var) )
  if var in categorical_cols:
    df_view.loc[df_view[var].isnull(),var]='SD'
    df_view.loc[df_view[var].isna(),var]='SD'

  if var in ordinal_cols:
    df_view.loc[df_view[var].isnull(),var]=9999
    df_view.loc[df_view[var].isna(),var]=9999
    
  y = df_view[(df_view['flag_train'] ==1)].default 
  X = df_view[(df_view['flag_train'] ==1)][var].values
  
  try: 
    optb = OptimalBinning(name=var, dtype="categorical",solver="cp", max_pvalue=.2 )
    optb.fit(X, y)

    tbla=optb.binning_table.build()
    tbla['Bin']=tbla.Bin.apply( lambda x: str(x)).astype('string')
    tbla=tbla[tbla.Bin.isin(['Missing', 'Special', ''])==False]
    #display(tbla)
    cuts_bivar_cat[var] = tbla.iloc[:,:6]

    #applies cuts in table
    
    nombre=str(var)+'_cat'

    l=list(zip(optb.splits, np.arange(0,len(optb.splits))))

    dic={}
    for l2 in l:
        for elem in l2[0]:
            dic[elem]=l2[1]
    
    df_view[nombre]=df_view[var].map(dic)
    imputa=df_view.loc[~df_view[nombre].isnull(),nombre].median()
    df_view.loc[df_view[nombre].isnull(),nombre]=imputa
    
    df_view[nombre] = df_view[nombre].astype(int)

    
    # nombre=str(var)+'_logit'
    # p=optb.transform(df_cons0[var], metric="event_rate")
    # df_cons0[nombre] = np.log(p/(1-p))
      

  except:
    print(var)
    print('Unexpected error:', sys.exc_info()[0])
    
    pass

var: x_13
var: x_14
var: x_18


In [9]:
limit_br=0.01*4
print('Print only if 1 value has positive rate greater than '+str(limit_br))
for key, value in cuts_bivar_cat.items():
  
  if (value.shape[0]>2) and (sum(value['Event rate']>limit_br)>0):
    print (key[0])
    print (key[1:])
    #print( value.to_string(index=False))
    display( value)
    print('\n ')

Print only if 1 value has positive rate greater than 0.04
m
erchant_category


,Bin,Count,Count (%),Non-event,Event,Event rate
0,['Prescription optics' 'Plants & Flowers' 'Di...,2163,0.120885,2156,7,0.003236
1,['Diversified entertainment' 'Diversified chil...,7367,0.411725,7325,42,0.005701
2,['Cosmetics' 'Children Clothes & Nurturing pro...,2210,0.123512,2185,25,0.011312
3,['Diversified electronics' 'Pharmaceutical pro...,2273,0.127033,2232,41,0.018038
4,['Youthful Shoes & Clothing' 'Digital services...,2928,0.163639,2859,69,0.023566
5,['Adult Shoes & Clothing' 'Electronic equipmen...,952,0.053205,895,57,0.059874



 
m
erchant_group


,Bin,Count,Count (%),Non-event,Event,Event rate
0,['Entertainment' 'Children Products'],9588,0.535852,9528,60,0.006258
1,['Health & Beauty' 'Jewelry & Accessories' 'Au...,1696,0.094786,1671,25,0.014741
2,"['Leisure, Sport & Hobby' 'Clothing & Shoes' '...",5713,0.319287,5598,115,0.020130
3,['Home & Garden' 'Intangible products' 'Food &...,896,0.050075,855,41,0.045759



 
a
ccount_worst_status_0_3m


...

,Bin,Count,Count (%),Non-event,Event,Event rate
0,[1.],5395,0.301515,5349,46,0.008526
1,[9999.],10921,0.610350,10800,121,0.011080
2,[2. 3. 4.],1577,0.088135,1503,74,0.046925



 
s
tatus_last_archived_0_24m


,Bin,Count,Count (%),Non-event,Event,Event rate
0,[1],12796,0.715140,12724,72,0.005627
1,[0],3418,0.191024,3326,92,0.026916
2,[2 3],1679,0.093836,1602,77,0.045861



 
s
tatus_max_archived_0_24_months


,Bin,Count,Count (%),Non-event,Event,Event rate
0,[5 1],8466,0.473146,8422,44,0.005197
1,[2],4725,0.264070,4688,37,0.007831
2,[0],3418,0.191024,3326,92,0.026916
3,[3],1284,0.071760,1216,68,0.052960


In [9]:
cuts_bivar_num={}
for var in numerical_cols:
  uniques= set(df_view[ (df_view.random<.1) &  (pd.isnull(df_view[var])==False)][var])

  print('var: ' + str(var) + ' - values null: '+  str(sum(df_view[var].isnull())) +
        '-  dif values: ' + str(len(uniques)))
  
  
  #xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx correct imputation
    #df_view[col]=df_view[col].fillna(0)
  y=df_view[(df_view['flag_train'] ==1)].default 
  X=df_view[(df_view['flag_train'] ==1)][var].values
  optb = OptimalBinning(name=var, 
    dtype="numerical", 
    solver="cp"
    ) 
    #,
    #monotonic_trend = "auto_asc_desc")#, monotonic_trend ='ascending')
      
  optb.fit(X, y) ##deberia pasarle solo los periodos de train

  tbla=optb.binning_table.build()
  #tbla=tbla[tbla.Bin.isin(['Missing', 'Special', ''])==False]
  tbla=tbla[tbla.Bin.isin(['Special'])==False]
  #display(tbla)
  cuts_bivar_num[var] = tbla.iloc[:,:6]


var: x_4 - values null: 53357-  dif values: 2486
var: x_37 - values null: 0-  dif values: 2190
var: x_6 - values null: 48934-  dif values: 4
var: x_19 - values null: 20658-  dif values: 291
var: x_17 - values null: 0-  dif values: 3808
var: x_39 - values null: 0-  dif values: 8428
var: x_32 - values null: 0-  dif values: 4
var: x_20 - values null: 0-  dif values: 25
var: x_28 - values null: 0-  dif values: 62
var: x_2 - values null: 10683-  dif values: 169
var: x_22 - values null: 0-  dif values: 8
var: x_16 - values null: 0-  dif values: 3565
var: x_40 - values null: 62540-  dif values: 3
var: x_7 - values null: 60055-  dif values: 4
var: x_23 - values null: 0-  dif values: 142
var: x_24 - values null: 0-  dif values: 129
var: x_30 - values null: 0-  dif values: 4
var: x_33 - values null: 0-  dif values: 5
var: x_1 - values null: 10683-  dif values: 19
var: x_21 - values null: 0-  dif values: 9
var: x_10 - values null: 0-  dif values: 71
var: x_38 - values null: 0-  dif values: 6022
v

In [12]:
limit_br=0.014*4
print('Print only if 1 value has positive rate greater than '+str(limit_br))
for key, value in cuts_bivar_num.items():
  
  if (value.shape[0]>2) and (sum(value['Event rate']>limit_br)>0):
    print (key[0])
    print (key[1:])
    #print( value.to_string(index=False))
    display( value)
    print('\n ')

Print only if 1 value has positive rate greater than 0.056
x
_11


,Bin,Count,Count (%),Non-event,Event,Event rate
0,"[-inf, 10.99)",3496,0.196736,3490,6,0.001716
1,"[10.99, 14.83)",3301,0.185763,3294,7,0.002121
2,"[14.83, 19.50)",2670,0.150253,2660,10,0.003745
3,"[19.50, 22.98)",1143,0.064322,1135,8,0.006999
4,"[22.98, 37.90)",2056,0.115701,2031,25,0.012160
5,"[37.90, inf)",890,0.050084,835,55,0.061798
7,Missing,4214,0.237141,4093,121,0.028714
Totals,,17770,1.000000,17538,232,0.013056



 
x
_34


,Bin,Count,Count (%),Non-event,Event,Event rate
0,"[-inf, 0.50)",3342,0.188070,3247,95,0.028426
1,"[0.50, 1.50)",8341,0.469387,8307,34,0.004076
2,"[1.50, 2.50)",4812,0.270793,4781,31,0.006442
3,"[2.50, inf)",1275,0.071750,1203,72,0.056471
5,Missing,0,0.000000,0,0,0.000000
Totals,,17770,1.000000,17538,232,0.013056


In [13]:
categorical_cols_cat = [str(x)+'_cat' for x in categorical_cols]

In [14]:


#print(categorical_cols)
predictors = numerical_cols + categorical_cols + categorical_cols_cat
end_num = len(numerical_cols)
end_cat = len(predictors)

In [15]:

l=predictors.copy()
l.extend(['default', 'flag_train'])
print(l)
df_view2 = df_view[l].copy()
cat_features = [df_view2.columns.get_loc(c) for c in categorical_cols + categorical_cols_cat if c in df_view2]

['x_4', 'x_37', 'x_6', 'x_19', 'x_17', 'x_39', 'x_32', 'x_20', 'x_28', 'x_2', 'x_22', 'x_16', 'x_40', 'x_7', 'x_23', 'x_24', 'x_30', 'x_33', 'x_1', 'x_21', 'x_10', 'x_38', 'x_12', 'x_9', 'x_11', 'x_15', 'x_36', 'x_25', 'x_8', 'x_27', 'x_31', 'x_26', 'x_34', 'x_5', 'x_0', 'x_29', 'x_3', 'x_35', 'id', 'x_13', 'x_14', 'x_18', 'x_13_cat', 'x_14_cat', 'x_18_cat', 'default', 'flag_train']
